In [ ]:
import itertools
n = 6  #number of symbols
UB = 871 #you have to put a target upperbound
permutations = list(itertools.permutations(list(range(n)), n))
print(permutations)

In [ ]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()
solver = cp_model.CpSolver()
assignment = {}
hasPerm = {}
num_vals = n
for s in range (0,n):
    for i in range(0,UB):
        assignment[(s, i)] = model.NewBoolVar("symbol %i is in position %i" % (s, i))
for p in range(0,len(permutations)):
    for i in range(0,UB-n+1):
        hasPerm[(p,i)] = model.NewBoolVar("permutation %i , %s is in position %i" % (p, permutations[p], i))

In [ ]:
#fix first permutation
#model.Add(hasPerm[(0,0)] == 1)

In [ ]:
# only one symbol per position
for i in range(0,UB):
    vars = []
    for s in range (0,n):
        vars.append(assignment[(s, i)])
    model.Add(sum(vars) == 1)

In [ ]:
# at most one permutation per position
for i in range(0,UB-n+1):
    vars = []
    for p in range (0,len(permutations)):
        vars.append(hasPerm[(p,i)])
    model.Add(sum(vars) <= 1)

In [ ]:
# each permutation at least once
for p in range (0,len(permutations)):
    vars = []
    for i in range(0,UB-n+1):
        vars.append(hasPerm[(p,i)])
    model.Add(sum(vars) >= 1)

In [ ]:
# link assignment to has permutation vars
for i in range(0,UB-n+1):
    for p in range (0,len(permutations)):
        perm = permutations[p]
        for shift in range(0,n):
            val = perm[shift]
            model.AddBoolOr([assignment[(val, i+shift)],hasPerm[(p,i)].Not()])

In [ ]:
#solve the model
status = solver.Solve(model)

In [ ]:
if status == cp_model.FEASIBLE:
    print("Feasible")
    print("Permutation:")  
    for i in range(0,UB):
        for s in range (0,n):
            val = solver.Value(assignment[(s, i)])
            if (val == 1):
                print(s,end='')
    print("")
    for i in range(0,UB-n+1):
        for p in range(0,len(permutations)):
            val = solver.Value(hasPerm[(p,i)])
            if (val == 1):
                print(hasPerm[(p,i)])
elif status == cp_model.INFEASIBLE:
    print("Not feasible")

